In [47]:
import sys
sys.path.insert(0, 'driftpy/src/')

import driftpy

from driftpy.math.trade import *
from driftpy.math.positions import *
from driftpy.math.amm import *
from driftpy.math.market import *

from driftpy.types import *
from driftpy.constants.numeric_constants import *

from programs.clearing_house.math.pnl import *
from programs.clearing_house.math.amm import *
from programs.clearing_house.state import *
from programs.clearing_house.lib import * 
from programs.clearing_house.state import * 
from sim.events import * 

import numpy as np 
import pandas as pd

import unittest

#%%
def default_set_up(n_users=1):
    length = 10 
    funding_period = 60 # every 60 ts 
    
    prices = [.5] * length # .5$ oracle 
    timestamps = np.arange(length)
    oracle = Oracle(prices=prices, timestamps=timestamps)

    # mark price = 1$ 
    amm = AMM(
        oracle=oracle, 
        base_asset_reserve=1_000_000 * AMM_RESERVE_PRECISION, 
        quote_asset_reserve=1_000_000 * AMM_RESERVE_PRECISION,
        peg_multiplier=1 * PEG_PRECISION, 
        funding_period=funding_period
    )
    market = Market(amm)
    fee_structure = FeeStructure(numerator=1, denominator=100)
    clearing_house = ClearingHouse([market], fee_structure)        

    for index in range(n_users):
        clearing_house = (
            clearing_house.deposit_user_collateral(
                user_index=index, 
                collateral_amount=10_000_000 * QUOTE_PRECISION
            )
        )
    return clearing_house    
        
ch: ClearingHouse = default_set_up(n_users=4)

In [48]:
ch = ch.add_liquidity(
    0, 0, 1_000_000 * QUOTE_PRECISION
)

In [49]:
ch.users[0].lp_positions[0]

LPPosition(market_index=0, lp_tokens=1000000000000, last_cumulative_lp_funding=0, last_net_position=0, last_fee_amount=0, last_quote_asset_amount=10000000000000000000)

In [50]:
# user takes all of amm's tokens
ch.markets[0].amm.lp_tokens == 0

True

In [51]:
market = ch.markets[0]
user = ch.users[1]
lp = ch.users[0]

calculate_mark_price(market)

1.0

In [52]:
# new user goes long 
ch = ch.open_position(
    PositionDirection.LONG, 
    1, 
    10_000 * QUOTE_PRECISION, 
    0
)

In [53]:
user_position = user.positions[0]
user_pnl = calculate_position_pnl(market, user_position)
user_pnl

0.0

In [54]:
user.collateral + lp.collateral # fees 

19999900000000.0

In [55]:
calculate_mark_price(market)

1.0201

In [56]:
user_position

MarketPosition(market_index=0, base_asset_amount=9.900990099009946e+16, quote_asset_amount=10000000000, last_cumulative_funding_rate=0, last_funding_rate_ts=0)

In [57]:
ch = ch.remove_liquidity(
    0, 0, lp.lp_positions[0].lp_tokens
)

In [58]:
calculate_mark_price(market)

1.0

In [59]:
market = ch.markets[0]
lp_position = ch.users[0].positions[0]
lp_pnl = calculate_position_pnl(market, lp_position)
lp_pnl

-1.0

In [60]:
market.amm.quote_asset_reserve, market.amm.base_asset_reserve, market.amm.net_base_asset_amount

(1e+19, 1e+19, 0.0)

In [61]:
# user1 is in profit 
market = ch.markets[0]
user = ch.users[1]
user_position = user.positions[0]
user_pnl = calculate_position_pnl(market, user_position)
user_pnl

-196078431.37249374

In [62]:
calculate_mark_price(market)

1.0

In [63]:
lp_position

MarketPosition(market_index=0, base_asset_amount=-9.900990099009946e+16, quote_asset_amount=10000000000, last_cumulative_funding_rate=0, last_funding_rate_ts=0)

In [64]:
lp_position.quote_asset_amount / user_position.quote_asset_amount

1.0

In [65]:
lp_position.base_asset_amount / user_position.base_asset_amount

-1.0

In [66]:
puser, plp = user.collateral, lp.collateral
puser, plp, puser + plp

(9999900000000.0, 10000100000000.0, 20000000000000.0)

In [67]:
# close user 
ch = ch.close_position(
    1, 0 
)
market = ch.markets[0]
user.collateral - puser

-294117647.05859375

In [68]:
# close LP
ch = ch.close_position(
    0, 0 
)
market = ch.markets[0]
lp.collateral - plp

98039214.67773438

In [69]:
user.collateral, lp.collateral

(9999605882352.941, 10000198039214.678)

In [34]:
market.amm.base_asset_reserve, market.amm.quote_asset_reserve, market.amm.net_base_asset_amount

(1e+19, 1e+19, 0.0)

In [70]:
# slightly less than the original amount bc paying for fees to close positions
(user.collateral + lp.collateral) / 1e6

19999803.92156762